In [3]:
import mysql.connector

In [4]:
from mysql.connector.pooling import MySQLConnectionPool

In [5]:
from mysql.connector import (connection, pooling, Error)

In [8]:
try:
    connection_config_dict = {
        'host': 'localhost',
        'user': 'root',
        # 'password': 'vicstrokes11',  # Remove the comment to include password
        'database': 'little_lemon_db',
        'autocommit': True
    }

    # Corrected the way of creating connection
    connection = mysql.connector.connect(**connection_config_dict)

    # Corrected the import of MySQLConnectionPool and usage
    pool = pooling.MySQLConnectionPool(pool_name="little_lemon_pool",
                                       pool_size=2,
                                       **connection_config_dict)

    print("Print the users that are requesting for connection.............")
    users = ['Anna', 'Joakim']
    select_stmt = "SELECT * FROM Bookings WHERE BookingID=%(booking_id)s"

    for i in range(pool.pool_size):
        connection = pool.get_connection()

        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()

            # Global connection timeout arguments
            global_connect_timeout = 'SET GLOBAL connect_timeout=180'
            global_wait_timeout = 'SET GLOBAL connect_timeout=180'
            global_interactive_timeout = 'SET GLOBAL connect_timeout=180'

            cursor.execute(global_connect_timeout)
            cursor.execute(global_wait_timeout)
            cursor.execute(global_interactive_timeout)

            connection.commit()

            # Requesting for connections for the customers
            print("The connection id for {} {} is requesting info on the booking {}"
                  .format(users[i], connection.connection_id, i + 1))
            cursor.execute(select_stmt, {'booking_id': i + 1})
            print(cursor.fetchall())

            # Creating the database and executing the database
            cursor.execute("USE little_lemon_db")
            print("Using little_lemon_db")

            # Creating a procedure query
            print("Creating a procedure query......")

            booking_procedure_query = """
            CREATE PROCEDURE PeakHours()
            BEGIN
                SELECT Bookings.BookingID,
                COUNT(HOUR(BookingSlot)) 
                FROM Bookings
                GROUP BY Bookings.TableNo DESC;
            END
            """

            guest_procedure_query = """
            CREATE PROCEDURE GuestStatus()
            BEGIN
            SELECT Bookings.BookingID,
            CONCAT(Bookings.GuestFirstName, ' ', Bookings.GuestLastName) AS CustomerName,
            Employees.Name, Employees.Role,
            CASE Employees.Role
                WHEN 'Manager' THEN 'Guest Order Status: Ready to Pay'
                WHEN 'Assistant Manager' THEN 'Guest Order Status: Ready to Pay'
                WHEN 'Head Chef' THEN 'Status: Ready to serve'
                WHEN 'Assistant Chef' THEN 'Status: Preparing Order'
                WHEN 'Head Waiter' THEN 'Status: Order served'
                ELSE 'Unknown'
            END AS Status
            FROM Bookings
            LEFT JOIN Employees ON Bookings.EmployeeID=Employees.EmployeeID;
            END
            """

            cursor.execute(booking_procedure_query)
            cursor.execute(guest_procedure_query)

            # Call procedures
            cursor.callproc("PeakHours")
            cursor.callproc("GuestStatus")

            # Fetch and print results
            for result in cursor.stored_results():
                print(result.fetchall())
        else:
            print("No live connection made")

except Error as e:
    print("Error while connecting to MySQL", e)

finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")


Print the users that are requesting for connection.............
Connected to MySQL Server version  5.5.5-10.4.28-MariaDB
The connection id for Anna 78 is requesting info on the booking 1
[(1, 12, 'Anna', 'Iversen', datetime.timedelta(seconds=68400), 1)]
Using little_lemon_db
Creating a procedure query......
Error while connecting to MySQL 1304 (42000): PROCEDURE PeakHours already exists
MySQL connection is closed
